In [7]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os
import sys
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [79]:
import torch
from sv_system.dnn.train.si_train import set_seed
from sv_system.dnn.utils import secToSample, secToSample
import torch.nn.functional as F
from sv_system.dnn.data.dataloader import init_default_loader

In [10]:
from sv_system.dnn.utils.score_utils import compute_eer

In [14]:
from sv_system.dnn.utils.parser import get_sv_parser
options = get_sv_parser().parse_args(args=[])
options.n_dct_filters = 40
options.n_mels = 40
options.timeshift_ms = 100
options.data_folder = "/home/muncok/DL/dataset/SV_sets"
options.window_size= 0.025
options.window_stride= 0.010
options.cache_size = 32768

In [36]:
options.input_format = 'fbank'
options.input_clip = True
options.input_length = secToSample(1)
options.splice_frames = secToSample(0.1)//160+1

In [37]:
options.data_folder = "/home/muncok/DL/dataset/SV_sets/speech_commands/"

### SI_Model

In [18]:
from dnn.model.SpeechModel import SpeechResModel
model = SpeechResModel("res15", 10)
model.load_partial("models/voxc/si_train/full_train/si_voxc_res15_0.1s_full_fbank.pt")
model.cuda()
lda = None  # None means not using lda

loaded from models/voxc/si_train/full_train/si_voxc_res15_0.1s_full_fbank.pt


In [57]:
from sv_system.dnn.model.AuxModels import SimpleCNN
options.input = "models/commands/equal_num_102spk_dot1.pt"
options.model = "SimpleCNN"
model = SimpleCNN(1880, options.splice_frames)
model.load(options.input)
lda = pickle.load(open("models/lda/equal_num_102spk_dot1_splice_lda.pkl", "rb"))
# lda = None

loaded from models/commands/equal_num_102spk_dot1.pt


In [60]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (convb_1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convb_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convb_3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convb_4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Ba

## Command Trial

In [19]:
from sv_system.dnn.sv_score import embeds

In [68]:
options.data_folder = '/home/muncok/DL/dataset/SV_sets/speech_commands/'
enroll_df = pd.read_pickle('./trials/commands/final/equal_num_102spk_enroll.pkl')
pos_test_df = pd.read_pickle('./trials/commands/final/equal_num_102spk_pos_test.pkl')
neg_test_df = pd.read_pickle("./trials/commands/final/equal_num_102spk_neg_test.pkl")

In [69]:
enroll_spks = enroll_df.spk.unique().tolist()

user_spks = enroll_spks[:2]
enroll_spks = enroll_spks[2:]

### Embeddings

In [70]:
from sv_system.dnn.data.dataset import SpeechDataset

In [71]:
dataset = SpeechDataset(SpeechDataset.read_df(options.data_folder, enroll_df), "test", vars(options))
val_dataloader = init_default_loader(dataset)
enroll_embeddings, _ = embeds(options, val_dataloader, model, lda)

LDA is loaded


In [72]:
dataset = SpeechDataset(SpeechDataset.read_df(options.data_folder, pos_test_df), "test", vars(options))
val_dataloader = init_default_loader(dataset)
pos_embeddings, _ = embeds(options, val_dataloader, model, lda)

LDA is loaded


In [73]:
dataset = SpeechDataset(SpeechDataset.read_df(options.data_folder, neg_test_df), "test", vars(options))
val_dataloader = init_default_loader(dataset)
imposter_embeddings, _ = embeds(options, val_dataloader, model, lda)

LDA is loaded


### Word Not Seperated

In [108]:
n_average = 1 # number of average means the number of words in a sentence
n_enroll_uttrs = 40 # number of enroll uttrs

In [109]:
spk_models = dict()
for spk in enroll_spks:
    enroll_idx = np.nonzero(enroll_df.spk == spk)
    spk_models[spk] = enroll_embeddings[enroll_idx][:n_enroll_uttrs].mean(0, True)

In [111]:
# average embedding --> scoring

pos_scores = dict()
neg_scores = dict()

for spk in enroll_spks:
    pos_test_idx = np.nonzero((pos_test_df.spk == spk))
    pos_embeds = pos_embeddings[pos_test_idx]
    pos_scores[spk] = F.cosine_similarity(pos_embeds,
                                          spk_models[spk])
   
    neg_scores[spk] = F.cosine_similarity(imposter_embeddings, spk_models[spk])

# for spk in enroll_spks:
#     print(spk)
#     compute_eer(pos_scores[spk], neg_scores[spk])
print("\nuni")
uni_pos_scores = np.concatenate([v for v in pos_scores.values()])
uni_neg_scores = np.concatenate([v for v in neg_scores.values()])
compute_eer(uni_pos_scores, uni_neg_scores)           


uni
eer:3.469, thres:0.3095


### Test on user speakers

In [407]:
n_enroll_uttrs = 40

In [420]:
spk_models = dict()
thres = 0.7
for spk in user_spks:
    enroll_idx = np.nonzero(enroll_df.spk == spk)
    spk_models[spk] = enroll_embeddings[enroll_idx][:n_enroll_uttrs].mean(0, True)

In [421]:
pos_scores = dict()
neg_scores = dict()
additional_embeds = {spk:[] for spk in user_spks}
for spk in user_spks:
    pos_embeds = pos_embeddings[np.nonzero(pos_test_df.spk == spk)]
    pos_scores[spk] = F.cosine_similarity(pos_embeds,
                                          spk_models[spk])
    additional_embeds[spk].append(pos_embeds[pos_scores[spk] > thres])    
#     print("additional_pos: {}".format(np.count_nonzero(pos_scores[spk] > thres)))
    
    neg_scores[spk] = F.cosine_similarity(imposter_embeddings, spk_models[spk])
    additional_embeds[spk].append(imposter_embeddings[neg_scores[spk] > thres])  
#     print("additional_neg: {}".format(np.count_nonzero(neg_scores[spk] > thres)))
    
    print(spk)
    compute_eer(pos_scores[spk], neg_scores[spk])
    
print("\nuni")
uni_pos_scores = np.concatenate([v for v in pos_scores.values()])
uni_neg_scores = np.concatenate([v for v in neg_scores.values()])
compute_eer(uni_pos_scores, uni_neg_scores)           

0ff728b5
eer:5.714, thres:0.3951

uni
eer:5.714, thres:0.3951


### provide additional info

In [422]:
for spk in user_spks:
    additional_spk_info = torch.cat(additional_embeds[spk])
    if len(additional_spk_info) == 0: continue
    print("spk: {} has {} additional_info".format(spk, len (additional_spk_info)))
    spk_models[spk] = (spk_models[spk] + additional_spk_info.mean(0, True)) / 2

spk: 0ff728b5 has 2 additional_info


In [423]:
pos_scores = dict()
neg_scores = dict()

for spk in user_spks:
    pos_test_idx = np.nonzero((pos_test_df.spk == spk))
    pos_embeds = pos_embeddings[pos_test_idx]
    pos_scores[spk] = F.cosine_similarity(pos_embeds,
                                          spk_models[spk])
    neg_scores[spk] = F.cosine_similarity(imposter_embeddings, spk_models[spk])

    print(spk)
    compute_eer(pos_scores[spk], neg_scores[spk])
    
print("\nuni")
uni_pos_scores = np.concatenate([v for v in pos_scores.values()])
uni_neg_scores = np.concatenate([v for v in neg_scores.values()])
compute_eer(uni_pos_scores, uni_neg_scores)           

0ff728b5
eer:5.714, thres:0.4016

uni
eer:5.714, thres:0.4016
